In [55]:
import os 
import sys 
import numpy as np 
import random 
import cv2
import matplotlib.pyplot as plt
import pickle
import pylab as pl
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.cluster import KMeans


from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC

np.random.seed(42)

In [5]:
data_dir = './data/'
class_list = ['Batman', 'Spiderman', 'None']


Width = 256
Height = 512

In [6]:
def get_file_names(path):
    return [os.path.join(path,f) for f in os.listdir(path)]

In [61]:
image_paths = {}
for training_name in class_list:
    dir_= os.path.join(data_dir, training_name)
    class_path = get_file_names(dir_)
    image_paths[training_name] = class_path

In [62]:
label_dict = {'Batman':0, 'Spiderman':1, 'None':2}
image_data = []
for k, v in image_paths.items():
    for im in v: 
        image_data.append((im, label_dict[k]))

In [63]:
np.random.shuffle(image_data)
image_paths, labels = zip(*image_data)

In [76]:
des_list=[]
hog = cv2.HOGDescriptor()

# https://stackoverflow.com/questions/44972099/opencv-hog-features-explanation
# https://answers.opencv.org/question/86023/what-is-the-hog-descriptors-shape/
#

for path in image_paths:
    img = cv2.imread(path)
    resized_image = []
    for i in range(3): 
        resized_image.append(cv2.resize(img[:,:,i], (128,64)))
    resized_image = np.array(resized_image).reshape(128,64,3)
    #creating hog features 
    descriptor = hog.compute(resized_image)
    des_list.append((path,np.array(descriptor)))

libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile


In [77]:
path.split('/')[2]

'None'

In [82]:
reshaped_features = []
image_labels = []
BoW = []
BoW_labels = [] 
for i in range(len(des_list)): 
    temp_des = des_list[i][1]
    reshaped_features.append(temp_des.reshape(15*7,-1))
    image_labels.append(label_dict[des_list[i][0].split('/')[2]])
    for j in range(reshaped_features[i].shape[0]):
        BoW.append(reshaped_features[i][j])
        BoW_labels.append(des_list[i][0].split('/')[2])

In [83]:
BoW_array = np.array(BoW)

In [84]:
BoW_array.shape

(22575, 36)

In [166]:
def build_histogram(descriptor_list, cluster_alg):
    histogram = np.zeros(len(cluster_alg.cluster_centers_))
    cluster_result =  cluster_alg.predict(descriptor_list)
    for i in cluster_result:
        histogram[i] += 1.0
    return histogram

kmeans = KMeans(n_clusters = 256)
kmeans.fit(BoW_array)

preprocessed_image = []

for descriptor in reshaped_features:
    if (descriptor is not None):
        temp_features = []
        for piece in descriptor: 
            histogram = build_histogram(piece.reshape(1,-1), kmeans)  # getting histogram for each piece 
            temp_features.append(histogram)
        temp_features = np.sum(np.array(temp_features), 0)
        preprocessed_image.append(temp_features.reshape(-1,))

In [167]:
preprocessed_image[0]

array([0., 0., 2., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 0., 0., 0., 1., 0., 0., 3., 0., 0., 0., 0., 0., 3., 1., 0.,
       2., 1., 1., 1., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0.,
       1., 1., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 1., 1., 1., 0., 2., 0., 0., 0., 0., 0., 0., 0., 1.,
       0., 0., 2., 1., 0., 0., 0., 1., 0., 0., 0., 2., 0., 0., 1., 1., 0.,
       0., 0., 0., 3., 1., 0., 0., 1., 1., 1., 0., 0., 0., 0., 3., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 1., 0., 0., 2., 0., 0., 1., 2., 0., 0., 0., 0., 2., 1., 0.,
       1., 1., 1., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 2., 1., 1.,
       0., 0., 2., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1.,
       0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1.,
       0., 1., 0., 0., 4., 0., 0., 0., 1., 0., 0., 2., 1., 0., 1., 3., 0.,
       0., 0., 0., 0., 0.

In [168]:
x_train, x_test, y_train, y_test = train_test_split(preprocessed_image, image_labels,
                                                    test_size = .25, random_state = 18)


In [169]:
grid = { 
    'C': [2**(-5), 2**(-4), 2**(-3), 2**(-2), 2**(-1), 1, 2, 4 ,8 ,16],
    'kernel': ['linear', 'poly', 'rbf'],
}

# grid = { 
#     'C': [2**(-5),  2**(-3), 1, 4 ,16],
#     'kernel': ['linear', 'rbf'],
# }

In [170]:
svm_cv = GridSearchCV(estimator=SVC(), param_grid=grid, cv= 5)
svm_cv.fit(x_train, y_train)

GridSearchCV(cv=5, estimator=SVC(),
             param_grid={'C': [0.03125, 0.0625, 0.125, 0.25, 0.5, 1, 2, 4, 8,
                               16],
                         'kernel': ['linear', 'poly', 'rbf']})

In [171]:
print(svm_cv.best_params_)
best_param = svm_cv.best_params_

{'C': 0.03125, 'kernel': 'poly'}


In [172]:
pickle_file_name = 'best_param_limmited.pkl'
with open(pickle_file_name, 'wb') as handle:
    pickle.dump(svm_cv.best_params_, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [162]:
pickle_file_name = 'best_param_limmited.pkl'
if os.path.exists(pickle_file_name):
    with open(pickle_file_name, 'rb') as handle:
        best_param = pickle.load(handle)

In [173]:
svm_clf = SVC(**best_param)
svm_clf.fit(x_train, y_train)

SVC(C=0.03125, kernel='poly')

In [174]:
y_pred = svm_clf.predict(x_test)

In [175]:
accuracy_score(y_test, y_pred)

0.46296296296296297